In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os
from resnet import *
from torch.autograd import Variable
import time

In [5]:
print(torch.__version__)

0.3.0.post4


In [4]:


############################################### Demo ########################################

leaning_rate = 0.1
resume = False
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
log_interval = 100 # printing interval
epoch = 3 # total training epochs
torch.manual_seed(0) # fix the seed


# Data
print('==> Preparing and Preprocessing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Model
if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt-cifar.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building neural model..')
    net = ResNet10(block=BasicBlock, activation=F.relu, batch_norm=True, num_classes=10)

optimizer = optim.SGD(net.parameters(), lr=leaning_rate, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx%log_interval == 0 and batch_idx > 0:
            print ('Loss: %.3f' % (train_loss/(batch_idx+1)) )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print ('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving to checkpoint..')
        state = {
            'net': net,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt-cifar.t7')
        best_acc = acc

train_time = 0
for epoch in range(start_epoch, start_epoch+epoch):
    start_time = time.time()
    train(epoch)
    end_time = time.time()
    train_time += end_time - start_time
    test(epoch)
print ('total training time: %d seconds' % train_time)

==> Preparing and Preprocessing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building neural model..

Epoch: 0
Loss: 1.858
Loss: 1.745
Loss: 1.660
Loss: 1.509 | Acc: 46.110% (4611/10000)
Saving to checkpoint..

Epoch: 1
Loss: 1.265
Loss: 1.227
Loss: 1.188
Loss: 1.102 | Acc: 61.190% (6119/10000)
Saving to checkpoint..

Epoch: 2
Loss: 1.019
Loss: 0.999
Loss: 0.977
Loss: 1.037 | Acc: 63.620% (6362/10000)
Saving to checkpoint..
total training time: 1120 seconds


In [5]:

############################################### Cell for removing batch normalization ########################################

leaning_rate = 0.1
resume = False
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
log_interval = 100 # printing interval
epoch = 3 # total training epochs
torch.manual_seed(0) # fix the seed


# Data
print('==> Preparing and Preprocessing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Model
if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt-cifar.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building neural model..')
    ###################################### fill following line ############################
    net = ResNet10(block=BasicBlock, activation=F.relu, batch_norm=False, num_classes=10)

optimizer = optim.SGD(net.parameters(), lr=leaning_rate, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx%log_interval == 0 and batch_idx > 0:
            print ('Loss: %.3f' % (train_loss/(batch_idx+1)) )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print ('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving to checkpoint..')
        state = {
            'net': net,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt-cifar.t7')
        best_acc = acc

train_time = 0
for epoch in range(start_epoch, start_epoch+epoch):
    start_time = time.time()
    train(epoch)
    end_time = time.time()
    train_time += end_time - start_time
    test(epoch)
print ('total training time: %d seconds' % train_time)

==> Preparing and Preprocessing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building neural model..

Epoch: 0
Loss: 2.208
Loss: 2.129
Loss: 2.063
Loss: 1.760 | Acc: 32.740% (3274/10000)
Saving to checkpoint..

Epoch: 1
Loss: 1.720
Loss: 1.704
Loss: 1.670
Loss: 1.503 | Acc: 44.410% (4441/10000)
Saving to checkpoint..

Epoch: 2
Loss: 1.501
Loss: 1.483
Loss: 1.459
Loss: 1.273 | Acc: 53.190% (5319/10000)
Saving to checkpoint..
total training time: 950 seconds


In [7]:
############################################### Cell for changing the activation function ########################################

leaning_rate = 0.1
resume = False
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
log_interval = 100 # printing interval
epoch = 3 # total training epochs
torch.manual_seed(0) # fix the seed


# Data
print('==> Preparing and Preprocessing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Model
if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt-cifar.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building neural model..')
    ###################################### fill following line ############################
    net = ResNet10(block=BasicBlock, activation=F.sigmoid, batch_norm=True, num_classes=10)

optimizer = optim.SGD(net.parameters(), lr=leaning_rate, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx%log_interval == 0 and batch_idx > 0:
            print ('Loss: %.3f' % (train_loss/(batch_idx+1)) )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print ('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving to checkpoint..')
        state = {
            'net': net,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt-cifar.t7')
        best_acc = acc

train_time = 0
for epoch in range(start_epoch, start_epoch+epoch):
    start_time = time.time()
    train(epoch)
    end_time = time.time()
    train_time += end_time - start_time
    test(epoch)
print ('total training time: %d seconds' % train_time)

==> Preparing and Preprocessing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building neural model..

Epoch: 0
Loss: 2.461
Loss: 2.352
Loss: 2.280
Loss: 2.471 | Acc: 20.050% (2005/10000)
Saving to checkpoint..

Epoch: 1
Loss: 1.952
Loss: 1.961
Loss: 1.944
Loss: 2.265 | Acc: 23.440% (2344/10000)
Saving to checkpoint..

Epoch: 2
Loss: 1.822
Loss: 1.824
Loss: 1.808
Loss: 2.922 | Acc: 20.170% (2017/10000)
total training time: 1144 seconds


In [3]:


############################################### Cell for changing to BottleneckBlock ########################################

leaning_rate = 0.1
resume = False
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
log_interval = 100 # printing interval
epoch = 3 # total training epochs
torch.manual_seed(0) # fix the seed


# Data
print('==> Preparing and Preprocessing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Model
if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt-cifar.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('==> Building neural model..')
    net = ResNet10(block=BottleneckBlock, activation=F.relu, batch_norm=True, num_classes=10)

optimizer = optim.SGD(net.parameters(), lr=leaning_rate, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        if batch_idx%log_interval == 0 and batch_idx > 0:
            print ('Loss: %.3f' % (train_loss/(batch_idx+1)) )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print ('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving to checkpoint..')
        state = {
            'net': net,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt-cifar.t7')
        best_acc = acc

train_time = 0
for epoch in range(start_epoch, start_epoch+epoch):
    start_time = time.time()
    train(epoch)
    end_time = time.time()
    train_time += end_time - start_time
    test(epoch)
print ('total training time: %d seconds' % train_time)

==> Preparing and Preprocessing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building neural model..

Epoch: 0
Loss: 1.962
Loss: 1.837
Loss: 1.755
Loss: 1.537 | Acc: 43.940% (4394/10000)
Saving to checkpoint..

Epoch: 1
Loss: 1.458
Loss: 1.434
Loss: 1.409
Loss: 1.367 | Acc: 51.070% (5107/10000)
Saving to checkpoint..

Epoch: 2
Loss: 1.274
Loss: 1.262
Loss: 1.245
Loss: 1.361 | Acc: 52.720% (5272/10000)
Saving to checkpoint..
total training time: 807 seconds
